In [1]:
%pip install -q -U transformers accelerate datasets bitsandbytes trl unsloth
%pip install -q -U evaluate rouge_score bert_score sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.6 MB/s eta 0:00:00


In [2]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/llama-3.1-8b-bnb-4bit"
max_seq_length = 2048
dtype = None
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Tải mô hình và tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.9.2: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [4]:
# Cấu hình LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    max_seq_length = max_seq_length,
)

Unsloth 2025.9.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import evaluate
from tqdm.auto import tqdm

In [6]:
# Tải và định dạng dataset ViHealthQA
print("Đang tải dataset tarudesu/ViHealthQA...")
dataset1 = load_dataset("tarudesu/ViHealthQA")

Đang tải dataset tarudesu/ViHealthQA...


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

val.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/7009 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/993 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2013 [00:00<?, ? examples/s]

In [7]:
print(dataset1['train'][0])

{'id': 1, 'question': 'Đang chích ngừa viêm gan B có chích ngừa Covid-19 được không?', 'answer': 'Nếu anh/chị đang tiêm ngừa vaccine phòng bệnh viêm gan B, anh/chị vẫn có thể tiêm phòng vaccine phòng Covid-19, tuy nhiên vaccine Covid-19 phải được tiêm cách trước và sau mũi vaccine viêm gan B tối thiểu là 14 ngày.', 'link': 'https://vnexpress.net/tu-van-tiem-vaccine-covid-19-an-toan-hieu-qua-p57'}


In [8]:
def format_instruction_vihealthqa(sample):
    return f"""### Question:\n{sample['question']}\n\n### Answer:\n{sample['answer']}EOS"""
formatted_dataset1 = dataset1['train'].map(lambda sample: {'text': format_instruction_vihealthqa(sample)}, remove_columns=dataset1['train'].column_names)


Map:   0%|          | 0/7009 [00:00<?, ? examples/s]

In [9]:
# Chia tập dữ liệu ViHealthQA thành train và test
train_test_split1 = formatted_dataset1.train_test_split(test_size=0.1)
train_dataset1 = train_test_split1['train']
test_dataset1 = train_test_split1['test']


In [10]:
# Khởi tạo SFTTrainer cho dataset ViHealthQA
trainer1 = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset1,
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "outputs_vihealthqa",
        optim = "adamw_8bit",
        seed = 3407,
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/6308 [00:00<?, ? examples/s]

In [11]:
# Huấn luyện mô hình
print("Bắt đầu fine-tuning trên dataset ViHealthQA...")
trainer1.train()

Bắt đầu fine-tuning trên dataset ViHealthQA...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,308 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: caonguyenthanhain (caonguyenthanhain-utel-universidad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.247600
2,2.162000
3,2.470300
4,2.557400
5,2.080400
6,2.261900
7,2.184000
8,2.177300
9,2.006500
10,2.006600


TrainOutput(global_step=60, training_loss=2.0327097713947295, metrics={'train_runtime': 446.7486, 'train_samples_per_second': 1.074, 'train_steps_per_second': 0.134, 'total_flos': 5164412137734144.0, 'train_loss': 2.0327097713947295, 'epoch': 0.07609384908053266})

In [12]:
# Đánh giá mô hình sau lần fine-tuning đầu tiên
print("\nĐang đánh giá mô hình trên dataset ViHealthQA...")
predictions1 = []
references1 = []

for sample in tqdm(test_dataset1):
    prompt = sample['text'].split('### Answer:')[0]
    reference_answer = sample['text'].split('### Answer:')[1]

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_answer = generated_text.split('### Answer:')[1]

    predictions1.append(generated_answer)
    references1.append([reference_answer])



Đang đánh giá mô hình trên dataset ViHealthQA...


  0%|          | 0/701 [00:00<?, ?it/s]

In [13]:
bleu_metric = evaluate.load("bleu")
bleu_results1 = bleu_metric.compute(predictions=predictions1, references=references1)
rouge_metric = evaluate.load("rouge")
rouge_results1 = rouge_metric.compute(predictions=predictions1, references=references1)
bertscore_metric = evaluate.load("bertscore")
bertscore_results1 = bertscore_metric.compute(predictions=predictions1, references=references1, lang="vi", device="cuda" if torch.cuda.is_available() else "cpu")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [14]:
print("\nKết quả đánh giá trên ViHealthQA:")
print("BLEU:", bleu_results1['bleu'])
print("ROUGE:", rouge_results1)
print("BERTScore:", bertscore_results1)


Kết quả đánh giá trên ViHealthQA:
BLEU: 0.03402578833594963
ROUGE: {'rouge1': np.float64(0.4610988492692507), 'rouge2': np.float64(0.18593785591648204), 'rougeL': np.float64(0.26625176107765536), 'rougeLsum': np.float64(0.2662197211758415)}
BERTScore: {'precision': [0.6843934059143066, 0.6022855043411255, 0.4799933433532715, 0.6698172092437744, 0.7573906183242798, 0.6454869508743286, 0.7199515700340271, 0.6377725601196289, 0.6277133822441101, 0.590369462966919, 0.642744779586792, 0.5877168774604797, 0.7424290776252747, 0.5478672385215759, 0.562849760055542, 0.4689923822879791, 0.6423971652984619, 0.7087438106536865, 0.6744952201843262, 0.5340030193328857, 0.5736250877380371, 0.47514986991882324, 0.5966173410415649, 0.7025527358055115, 0.8013771772384644, 0.5986002087593079, 0.7556402683258057, 0.46302902698516846, 0.6280543208122253, 0.5610470771789551, 0.5185225009918213, 0.5265716314315796, 0.6442482471466064, 0.5567572116851807, 0.7268680334091187, 0.5282039642333984, 0.48248937726

In [15]:
# Lưu adapter LoRA đã huấn luyện vào thư mục 'lora_model'
model.save_pretrained("lora_model")

# (Tùy chọn) Lưu cả tokenizer đi kèm
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [16]:
!zip -r lora_model.zip lora_model

  adding: lora_model/ (stored 0%)
  adding: lora_model/special_tokens_map.json (deflated 71%)
  adding: lora_model/README.md (deflated 65%)
  adding: lora_model/adapter_model.safetensors (deflated 8%)
  adding: lora_model/adapter_config.json (deflated 57%)
  adding: lora_model/tokenizer.json (deflated 85%)
  adding: lora_model/tokenizer_config.json (deflated 96%)


In [17]:
from google.colab import files
files.download('lora_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import os

# Tên thư mục nơi bạn đã lưu adapter LoRA
# Dựa trên file notebook của bạn, thư mục này có tên là 'lora_model'.
adapter_dir = "lora_model"

# Tên mô hình gốc mà bạn đã sử dụng để huấn luyện
model_name = "unsloth/llama-3.1-8b-bnb-4bit"

# Kiểm tra xem GPU có sẵn không
if not torch.cuda.is_available():
    print("Vui lòng chọn môi trường chạy có GPU (ví dụ: Google Colab với GPU T4).")
else:
    print("GPU đã sẵn sàng. Bắt đầu tải mô hình...")

    # Tải mô hình gốc và tokenizer
    try:
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=model_name,
            max_seq_length=2048,
            dtype=None,
            load_in_4bit=True,
        )
    except NotImplementedError:
        print("Lỗi: Unsloth chỉ hoạt động với GPU NVIDIA hoặc Intel. Đảm bảo bạn đã chọn GPU phù hợp trong cài đặt runtime.")
        model = None

    if model:
        # Cấu hình LoRA (cần thiết để tải lại adapter)
        model = FastLanguageModel.get_peft_model(
            model,
            r=16,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                              "gate_proj", "up_proj", "down_proj",],
            lora_alpha=16,
            lora_dropout=0,
            bias="none",
            use_gradient_checkpointing="unsloth",
            random_state=3407,
            max_seq_length=2048,
        )

        # Định dạng câu hỏi cho mô hình
        def create_prompt(question):
            return f"""### Question:
{question}

### Answer:
"""

        # Vòng lặp hỏi đáp
        print("\nBắt đầu trò chuyện với mô hình. Gõ 'exit' để thoát.")
        while True:
            question = input("Bạn có câu hỏi nào về sức khỏe không? ")
            if question.lower() == 'exit':
                break

            prompt = create_prompt(question)

            # Mã hóa câu hỏi và tạo câu trả lời
            inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=512,
                    use_cache=True,
                    pad_token_id=tokenizer.eos_token_id,
                    do_sample=True,
                    top_k=50,
                    top_p=0.95
                )

            # Giải mã câu trả lời và in ra màn hình
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Xử lý trường hợp mô hình trả lời theo định dạng khác nhau
            if '### Answer:' in generated_text:
                generated_answer = generated_text.split('### Answer:')[1].strip().replace('EOS', '')
            elif '### Output:' in generated_text:
                generated_answer = generated_text.split('### Output:')[1].strip().replace('EOS', '')
            else:
                generated_answer = generated_text.strip()

            print("\n[Mô hình trả lời]:")
            print(generated_answer)
            print("-" * 50)

GPU đã sẵn sàng. Bắt đầu tải mô hình...
==((====))==  Unsloth 2025.9.2: Fast Llama patching. Transformers: 4.56.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

Bắt đầu trò chuyện với mô hình. Gõ 'exit' để thoát.
Bạn có câu hỏi nào về sức khỏe không? Bé 13 tháng tuổi uống thuốc Acyclovir có được không?

[Mô hình trả lời]:
Acyclovir là thuốc chống virus Herpes simplex và Varicella zoster (chủ yếu là herpes simplex loại 1, loại 2 và virus varicella). Acyclovir là thuốc kháng virus được sử dụng rộng rãi nhất, đặc biệt trong các bệnh về da. Acyclovir không gây độc tính đối với bào thai, nhưng có thể gây ra các tác dụng phụ như buồn nôn, tiêu chảy, đau bụng, số